In [ ]:
using Glob
using JSON
using LargeScaleAnalysis
using ProgressMeter
using PyPlot
using Statistics
using StatsBase
using ThesisTools

### Helpers

In [ ]:
DateFormatter = pyimport("matplotlib.dates").DateFormatter
HourLocator = pyimport("matplotlib.dates").HourLocator;

In [ ]:
using Dates
function PyPlot.plot(model::DataSegmentationModel; ax = gca())
    cmap = Dict(i => plt.cm.tab20(i) for i in sort(unique(model.state)))
    ax.plot(unix2datetime.(model.index), coalesce.(model.data, NaN))
    for segment in segments(model, closed = true)
        span = unix2datetime.((segment.range.start, segment.range.stop))
        ax.axvspan(span..., alpha = 0.3, color = cmap[segment.state])
    end
end

### Analysis of self measurements

In [ ]:
files = glob("*.model.json", "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_self_pairs/");

In [ ]:
models = map(x -> parsefile(DataSegmentationModel, x), files);

In [ ]:
nstates = map(x -> size(x.model, 1), models);

In [ ]:
nstatesdist = counts(nstates, maximum(nstates))
bar(1:length(nstatesdist), nstatesdist);

In [ ]:
# TODO: Check in traceroute that there is no "spurious" hops

In [ ]:
# "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_self_pairs/10509900_6323.ndjson.model.json"
@pywith plt.style.context("classicthesis") begin
    model, file = models[nstates .== 1][7], files[nstates .== 1][7]
    @show file
    fig, ax = subplots(figsize = (4.6, 0.35*4.6))
    plot(model, ax = ax)
    ax.set(ylabel = "RTT (ms)", title = "pl-ktw-as206186.anchors.atlas.ripe.net")
    ax.xaxis.set_major_formatter(DateFormatter("%d/%m\n%Hh%M"))
    ax.xaxis.set_major_locator(HourLocator(byhour=[12]))
    fig.savefig("../figures/atlas-self-example-1state.pdf")
end

In [ ]:
# "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_self_pairs/23154956_6642.ndjson.model.json"
@pywith plt.style.context("classicthesis") begin
    model, file = models[nstates .== 3][7], files[nstates .== 3][7]
    @show file
    fig, ax = subplots(figsize = (4.6, 0.35*4.6))
    plot(model, ax = ax)
    ax.set(ylabel = "RTT (ms)", title = "lu-lux-as199524.anchors.atlas.ripe.net")
    ax.xaxis.set_major_formatter(DateFormatter("%d/%m\n%Hh%M"))
    ax.xaxis.set_major_locator(HourLocator(byhour=[12]))
    fig.savefig("../figures/atlas-self-example-3state.pdf")
end

In [ ]:
# file = "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_self_pairs/22933861_6613.ndjson.model.json"
@pywith plt.style.context("classicthesis") begin
    model, file = models[nstates .== 5][1], files[nstates .== 5][1]
    @show file
    fig, ax = subplots(figsize = (4.6, 0.35*4.6))
    plot(model, ax = ax)
    ax.set(ylabel = "RTT (ms)", title = "us-cbf-as15169.anchors.atlas.ripe.net")
    ax.xaxis.set_major_formatter(DateFormatter("%d/%m\n%Hh%M"))
    ax.xaxis.set_major_locator(HourLocator(byhour=[12]))
    fig.savefig("../figures/atlas-self-example-5state.pdf")
end

### Analysis of non-self measurements

In [ ]:
# TODO: Re-do analysis with 10% of the pairs instead of 1%.
# TODO: ACF insides the states?

In [ ]:
# TODO: Number of states vs. number of hops ?
# TODO: Comparer modeles appris sur 1 et 3 jours vs. les sous-sequences
# de 1 et 3 jours sur un modele appris sur 7 jours (est-ce que ca match ?).

In [ ]:
# files = glob("*ndjson.model.json", "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_noself_pairs/");
files = glob("*ndjson.model.json", "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_01_pairs/");
files_1d = glob("*ndjson_360.model.json", "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_noself_pairs/");
files_3d = glob("*ndjson_1080.model.json", "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_noself_pairs/");
files_14d = glob("*ndjson.model.json", "/blobs/datasets/thesis/ping_v4_1580511600_1581721200_noself_pairs/");

In [ ]:
models = map(x -> parsefile(DataSegmentationModel, x), files);
models_1d = map(x -> parsefile(DataSegmentationModel, x), files_1d);
models_3d = map(x -> parsefile(DataSegmentationModel, x), files_3d);
models_14d = map(x -> parsefile(DataSegmentationModel, x), files_14d);

In [ ]:
nstates = map(x -> size(x.model, 1), models);
nstates_1d = map(x -> size(x.model, 1), models_1d);
nstates_3d = map(x -> size(x.model, 1), models_3d);
nstates_14d = map(x -> size(x.model, 1), models_14d);

In [ ]:
fig, ax = subplots()
ax.boxplot([nstates_1d, nstates_3d, nstates, nstates_14d], labels = ["1 jour", "3 jours", "7 jours", "14 jours"], showfliers = false)
ax.set(ylabel = "Nombre d'états", ylim = (0, 17))
ax.grid(axis = "x")
save_thesis("atlas_nstates_dist_boxplot", hwr = 0.5, extra_axis_params = ["xtick={1,2,3,4}", "xticklabels={1 jour, 3 jours, 7 jours, 14 jours}"])

In [ ]:
@time median.([nstates_1d, nstates_3d, nstates, nstates_14d])

In [ ]:
# Verifier si pour une series donnee le nombre d'etats augmente toujours avec le temps.
# Chercher les series pour lesquelles le nombre d'etats entre 7 et 14 jours change peu/change beaucoup.

In [ ]:
# Strange series :
# file = "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_noself_pairs/19647105_6672.ndjson.model.json"
# file = "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_noself_pairs/19649596_6175.ndjson.model.json"
# file = "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_01_pairs/19373231_6297.ndjson.model.json"
# file = "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_01_pairs/23739707_6691.ndjson.model.json"

In [ ]:
# Series with clear delay changes (routing?)
# file = "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_01_pairs/17828883_6497.ndjson.model.json"
# file = "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_01_pairs/22680685_6235.ndjson.model.json"

In [ ]:
# Clusters de transitions :
# file = "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_01_pairs/18724961_6441.ndjson.model.json"

In [ ]:
# @pywith plt.style.context("classicthesis") begin
# k = rand(1:20)
# k = 9
i = rand(1:length(models[nstates .== k]))
    model, file = models[nstates .== k][i], files[nstates .== k][i]
    @show file
    # fig, ax = subplots(figsize = (4.6, 0.35*4.6))
    fig, ax = subplots(figsize = (10, 3))
    plot(model, ax = ax)
    ax.set(
        xlim = (DateTime(2020, 2, 1), DateTime(2020, 2, 3)),
        ylabel = "RTT (ms)"
    )#, title = "us-cbf-as15169.anchors.atlas.ripe.net")
    ax.xaxis.set_major_formatter(DateFormatter("%d/%m\n%Hh%M"))
    ax.xaxis.set_major_locator(HourLocator(byhour=[12]))
#     fig.savefig("../figures/atlas-self-example-5state.pdf")
# end

In [ ]:
matshow(model.model.A)

In [ ]:
# file = "/blobs/datasets/thesis/ping_v4_1580511600_1581116400_noself_pairs/19647105_6672.ndjson.model.json"

#### Relation CV / durée

In [ ]:
files = glob("*ndjson.model.json", "../data/ping_v4_1580511600_1581116400_01_pairs/");
models = map(x -> parsefile(DataSegmentationModel, x), files);

In [ ]:
# Compute avg. durations by states
durations, stds = [], []
# for model in models_14d
for model in models
    for (state, segments) in group(segments(model.state))
        data = model.data[segments]
        push!(durations, mean(length, segments))
        push!(stds, std(skipmissing(data)) / mean(skipmissing(data)))
    end
end

durations = durations[isfinite.(stds)]
stds = stds[isfinite.(stds)];

In [ ]:
maximum(stds)

In [ ]:
# Δ = 500
# groups = []
# for i in 1:Δ:5000
#     indices = (durations .>= i) .& (durations .< i+Δ)
#     push!(groups, stds[indices])
# end;

In [ ]:
Δ = 0.2
I = []
groups = []
for i in 0:Δ:2
    indices = (stds .>= i) .& (stds .< i+Δ)
    if sum(indices) <= 10
        break
    end
    push!(I, i + Δ)
    push!(groups, durations[indices])
end;

In [ ]:
median(durations[stds .> 0.2]), median(durations[stds .< 0.2])

In [ ]:
fig, ax = subplots()
labels = map(x -> round(x, digits = 2), I)
ax.boxplot(groups, labels = labels, showfliers = false)
ax.set(xlabel = "Coefficient de variation", ylabel = "Durée moyenne (minutes)", ylim = (-10, 310))
ax.grid(axis = "x")
save_thesis("atlas_cv_duration_boxplot", hwr = 0.5, extra_axis_params = ["xtick={$(join(1:length(labels), ","))}", "xticklabels={$(join(labels, ", "))}"])

In [ ]:
safe(f) = try f(); catch NaN; end

In [ ]:
plot(collect(0:Δ:1), map(median, groups))
plot(collect(0:Δ:1), map(x -> percentile(x, 25), groups))
plot(collect(0:Δ:1), map(x -> percentile(x, 75), groups))
# xlim(-2, 200)

In [ ]:
# using Seaborn
# fig, ax = subplots()
# kdeplot(log.(durations), log.(stds), ax = ax, shade=true)
# # ax.set_xscale("log")
# # ax.set_yscale("log")